# UCL Round of 16 Draw: An Integer Linear Programming Approach
## Mohamad Chehade

In [18]:
import pandas as pd
import pulp as pl

In [19]:
g = 8 # number of groups
n = 8 # number of matches 

In [20]:
df = pd.read_excel (r'UCL Round of 16 Teams.xlsx')
df = df.drop(columns = "Group")
df.head()

,Leader,Leader Rating,Leader Country,Runner-up,Runner-up Rating,Runner-up Country
0,Napoli,3549,Italy,Liverpool,4065,England
1,Porto,3439,Portugal,Club Brugge,2671,Belgium
2,Bayern Munich,4201,Germany,Inter Milan,3552,Italy
3,Tottenham Hotspur,3439,England,Eintracht Frankfurt,3231,Germany
4,Chelsea,3603,England,Milan,3349,Italy


In [21]:
# create the two dictionaries for countries
 
D_leader = {}
D_runner_up = {}

for ind in df.index:
    element = df["Leader Country"][ind]
    value = df["Leader"][ind]
    if element in D_leader:
        D_leader[element].append(value)
    else:
        D_leader[element] = [value]

for ind in df.index:
    element = df["Runner-up Country"][ind]
    value = df["Runner-up"][ind]
    if element in D_runner_up:
        D_runner_up[element].append(value)
    else:
        D_runner_up[element] = [value]   

In [22]:
# create the two arrays for ratings

ratings_leader = df["Leader Rating"].tolist()
ratings_runner_up = df["Runner-up Rating"].tolist()

In [23]:
# create the optimization problem with the decision variables 

m = pl.LpProblem('UCL Round of 16 Draw', pl.LpMinimize)

teams = range(g)
matches = range(n)

x = pl.LpVariable.dicts(name="Leader", indices=(teams,matches),cat='Binary')
y = pl.LpVariable.dicts(name="RunnerUp", indices=(teams,matches),cat='Binary')

z = pl.LpVariable.dicts(name="Match_Sum", indices=(matches),lowBound=0, cat='Continuous')
U = pl.LpVariable.dicts(name="U", indices=(matches),lowBound=0, cat='Continuous')

z_bar = pl.LpVariable(name="Matches_Average", cat='Continuous')


In [24]:
# objective function

m += ( pl.lpSum([U[j] for j in matches]), 'Total Error') ;

In [25]:
# absolute value constraints

for j in matches:
    m += (z_bar - z[j] <= U[j])
    m += (z_bar - z[j] >= - U[j])

In [26]:
# define the variable z, the sum of the two ratings of a match

for j in matches:
    m += (z[j] == pl.lpSum(ratings_leader[i]*x[i][j] for i in teams) + pl.lpSum(ratings_runner_up[i]*y[i][j] for i in teams) )

In [27]:
# define the variable z_bar, the average of all sums z 

m += (z_bar == 1/n * (pl.lpSum(z[j] for j in matches)))

In [28]:
# one leader and one runner-up in each match 

for j in matches:
    m += (pl.lpSum(x[i][j] for i in teams)==1) # leader
    m += (pl.lpSum(y[i][j] for i in teams)==1) # runner_up

In [29]:
# each team plays in one match only

for i in teams:
    m += (pl.lpSum(x[i][j] for j in matches)==1) # leader
    m += (pl.lpSum(y[i][j] for j in matches)==1) # runner_up 

In [30]:
# teams from the same group cannot face each other

for j in matches:
    for i in teams:
        m += (x[i][j] + y[i][j] <= 1)

In [31]:
# teams from the same country cannot face each other

for j in matches:
    for element in D_leader:
        if element in D_runner_up:
            s = 0
            for k in range (len(D_leader[element])):
                for l in range (len(D_runner_up[element])):
                    m += x[k][j]+y[l][j] <=1

In [32]:
# solve

status = m.solve()
pl.LpStatus[status]

'Optimal'

In [33]:
# store results 

D_matches = {}

for v in m.variables():
    
    if v.varValue>=1:
    
        indicator = v.name[0]
        if indicator=="L" or indicator[0]=="R":
            keys = v.name.split("_")
            group = int(keys[1])
            match = int(keys[2])

            if indicator == "L":
                team = df["Leader"][group]    
            else: 
                team = df["Runner-up"][group]

            if match in D_matches:
                D_matches[match].append(team)
            else:
                D_matches[match] = [team]

In [34]:
for element in D_matches:
    print(D_matches[element][0],"vs",D_matches[element][1])

Napoli vs RB Leipzig
Porto vs Paris Saint-Germain
Bayern Munich vs Milan
Tottenham Hotspur vs Inter Milan
Chelsea vs Borussia Dortmund
Real Madrid vs Eintracht Frankfurt
Manchester City vs Club Brugge
Benfica vs Liverpool
